<a href="https://colab.research.google.com/github/nithinpadmanabhuni-eng/Hypothesis-Testing-on-Interest-Rates/blob/main/Hypothesis_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:

!pip install pandas numpy scipy


In [2]:
import pandas as pd
import numpy as np
from scipy import stats


In [7]:
import os

for f in os.listdir():
    print(f)


.config
Us-Interest Rate-Weekly.xlsx
sample_data


In [9]:
DATA_FILE = "/Us-Interest Rate-Weekly.xlsx"  # example: "Us-Interest Rate-Weekly.xlsx"

df = pd.read_excel(DATA_FILE)

print("Loaded Excel file successfully!")
print(df.head())

Loaded Excel file successfully!
   Unnamed: 0        Date  Value
0           0  1971-08-04   5.50
1           1  1971-08-15   5.50
2           2  1971-08-16   5.75
3           3  1971-08-31   5.75
4           4  1971-09-01   5.13


In [22]:

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
print("Columns after cleanup:", list(df.columns))

df = df.rename(columns={
    "Date": "DATE",
    "Value": "FEDFUNDS"
})

print("Columns after renaming:", list(df.columns))


df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce")


df = df.dropna(subset=["DATE", "FEDFUNDS"])

print("Rows after cleaning:", len(df))
df.head()


Columns after cleanup: ['DATE', 'FEDFUNDS']
Columns after renaming: ['DATE', 'FEDFUNDS']
Rows after cleaning: 1683


,DATE,FEDFUNDS
0,1971-08-04,5.50
1,1971-08-15,5.50
2,1971-08-16,5.75
3,1971-08-31,5.75
4,1971-09-01,5.13


In [12]:
# Period 1: 2010–2014
period1 = df[(df["DATE"] >= "2010-01-01") & (df["DATE"] <= "2014-12-31")]["FEDFUNDS"]

# Period 2: 2015–2019
period2 = df[(df["DATE"] >= "2015-01-01") & (df["DATE"] <= "2019-12-31")]["FEDFUNDS"]

print("Period 1 count (2010–2014):", len(period1))
print("Period 2 count (2015–2019):", len(period2))

period1.head(), period2.head()


Period 1 count (2010–2014): 122
Period 2 count (2015–2019): 120


(1335    0.25
 1336    0.25
 1337    0.25
 1338    0.25
 1339    0.25
 Name: FEDFUNDS, dtype: float64,
 1457    0.25
 1458    0.25
 1459    0.25
 1460    0.25
 1461    0.25
 Name: FEDFUNDS, dtype: float64)

In [19]:
from scipy import stats
import numpy as np

t_stat_for_z, _ = stats.ttest_ind(period1, period2, equal_var=True)

z_stat = t_stat_for_z
p_value_z = 2 * (1 - stats.norm.cdf(abs(z_stat)))

print("===== Z-TEST (approx) =====")
print("H0: mean FEDFUNDS is same in both periods")
print(f"Z statistic : {z_stat:.4f}")
print(f"P-value     : {p_value_z:.6f}")


===== Z-TEST (approx) =====
H0: mean FEDFUNDS is same in both periods
Z statistic : -13.1029
P-value     : 0.000000


/usr/local/lib/python3.12/dist-packages/scipy/stats/_axis_nan_policy.py:579: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


In [14]:
t_stat, p_value_t = stats.ttest_ind(period1, period2, equal_var=False)

print("===== T-TEST =====")
print("H0: mean FEDFUNDS is same in both periods")
print(f"t statistic : {t_stat:.4f}")
print(f"P-value     : {p_value_t:.6f}")


===== T-TEST =====
H0: mean FEDFUNDS is same in both periods
t statistic : -12.9946
P-value     : 0.000000


In [20]:
import numpy as np
from scipy import stats



var1 = np.var(period1, ddof=1)
var2 = np.var(period2, ddof=1)

print("Variance 2010–2014:", var1)
print("Variance 2015–2019:", var2)


if var1 == 0 or var2 == 0:
    print("One of the variances is zero. F-test cannot be performed meaningfully.")
else:

    if var1 >= var2:
        F = var1 / var2
        df1 = len(period1) - 1
        df2 = len(period2) - 1
        num = "2010–2014"
        den = "2015–2019"
    else:
        F = var2 / var1
        df1 = len(period2) - 1
        df2 = len(period1) - 1
        num = "2015–2019"
        den = "2010–2014"


    p_one_tail = 1 - stats.f.cdf(F, df1, df2)
    p_value_f = 2 * p_one_tail

    print("===== F-TEST =====")
    print("H0: variance of FEDFUNDS is same in both periods")
    print(f"F statistic ({num}/{den}) : {F:.4f}")
    print(f"P-value                 : {p_value_f:.6f}")


Variance 2010–2014: 0.0
Variance 2015–2019: 0.6526610644257703
One of the variances is zero. F-test cannot be performed meaningfully.


In [18]:
alpha = 0.05

print("===== SUMMARY (α = 0.05) =====")

# Z-test
if p_value_z < alpha:
    print("Z-test : Reject H0 → Mean FEDFUNDS is significantly different between 2010–2014 and 2015–2019.")
else:
    print("Z-test : Fail to reject H0 → No strong evidence of mean difference between periods (Z-test).")

# t-test
if p_value_t < alpha:
    print("T-test: Reject H0 → Mean FEDFUNDS is significantly different between the two periods.")
else:
    print("T-test: Fail to reject H0 → Means appear similar between the two periods (t-test).")

# F-test
if 'p_value_f' in globals():
    if isinstance(p_value_f, (int, float)):
        if p_value_f < alpha:
            print("F-test: Reject H0 → Variance of FEDFUNDS is significantly different between periods.")
        else:
            print("F-test: Fail to reject H0 → No strong evidence of variance difference.")
    else:
        print("F-test: Not computed due to zero variance.")
else:
    print("F-test: Not run.")


===== SUMMARY (α = 0.05) =====
Z-test : Reject H0 → Mean FEDFUNDS is significantly different between 2010–2014 and 2015–2019.
T-test: Reject H0 → Mean FEDFUNDS is significantly different between the two periods.
F-test: Fail to reject H0 → No strong evidence of variance difference.
